# Feature Extraction: Two-Stage Funnel Selection

This notebook performs **two-stage feature selection** to reduce high-dimensional genomic data to a manageable size for quantum machine learning:

## Pipeline Overview:
1. **Stage 1 - Mutual Information Filter**: Coarse sieve using univariate MI scores (50K features)
2. **Stage 2 - XGBoost Importance**: Fine-toothed comb using tree-based feature importance (500 features)
3. **NaN Validation**: Verify output data quality
4. **Packaging**: Copy indicators and create archive

## Input/Output:
- **Input**: `final_filtered_datasets/` (imputed modality parquets + indicator_features.parquet)
- **Output**: `final_processed_datasets_xgb_balanced/` (6 modality parquets @ 500 features each + indicators)

## Configuration & Utilities

In [ ]:
import pandas as pd
import os
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel, SelectKBest, mutual_info_classif
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

# ============================================================================
# CONFIGURATION - Adjust paths and parameters here
# ============================================================================
CONFIG = {
    # Paths (adjust for your environment)
    'source_dir': '/kaggle/input/data-process/quantum-classification/final_filtered_datasets',
    'output_dir': 'final_processed_datasets_xgb_balanced',
    'indicator_file': '/kaggle/input/data-process/quantum-classification/final_filtered_datasets/indicator_features.parquet',
    
    # Feature selection parameters
    'n_features_stage1': 50000,    # Mutual Info coarse filter
    'n_features_final': 500,       # XGBoost fine filter
    
    # Data modalities to process
    'modalities': ['CNV', 'GeneExpr', 'miRNA', 'Meth', 'Prot', 'SNV'],
    
    # Memory management
    'thrift_limit': 1 * 1024**3,   # 1GB for parquet loading
}

# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================
def safe_load_parquet(file_path, columns=None):
    """Load parquet file with increased thrift limit for large genomic data."""
    try:
        return pd.read_parquet(
            file_path,
            columns=columns,
            thrift_string_size_limit=CONFIG['thrift_limit'],
            thrift_container_size_limit=CONFIG['thrift_limit']
        )
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def ensure_dir(path):
    """Create directory if it doesn't exist."""
    os.makedirs(path, exist_ok=True)
    return path

# Create output directory
ensure_dir(CONFIG['output_dir'])
print(f"Configuration loaded. Output: {CONFIG['output_dir']}")

## Stage 1 & 2: Two-Stage Feature Selection

For each modality:
1. **Load data** and filter to non-missing cases for robust feature selection
2. **Mutual Information filter**: Reduce to top 50K features based on univariate MI scores
3. **XGBoost filter**: Train classifier, select top 500 features by importance
4. **Reload & save**: Reload only selected columns to minimize memory usage

In [ ]:
def apply_two_stage_selection(modality, indicators_df, label_encoder, master_sort_order):
    """
    Apply two-stage feature selection (MI -> XGBoost) to a single modality.
    
    Returns True if successful, False otherwise.
    """
    input_file = os.path.join(CONFIG['source_dir'], f'data_{modality}_.parquet')
    output_file = os.path.join(CONFIG['output_dir'], f'data_{modality}_.parquet')
    
    print(f"\n{'='*60}")
    print(f"Processing: {modality}")
    print(f"{'='*60}")
    
    # Load full dataset
    df_full = safe_load_parquet(input_file)
    if df_full is None:
        return False
    
    n_features = df_full.shape[1] - 2  # Exclude case_id and class
    print(f"  Original features: {n_features:,}")
    
    # Skip selection if already small enough
    if n_features <= CONFIG['n_features_final']:
        print(f"  -> Dataset already small enough. Sorting and saving directly.")
        df_full.set_index('case_id', inplace=True)
        df_sorted = df_full.reindex(master_sort_order).reset_index()
        df_sorted.to_parquet(output_file, index=False)
        return True
    
    # --- Filter to non-missing cases for robust feature selection ---
    is_missing_col = f'is_missing_{modality}_'
    case_ids_present = indicators_df[indicators_df[is_missing_col] == 0].index
    df_for_selection = df_full[df_full['case_id'].isin(case_ids_present)]
    del df_full
    
    X = df_for_selection.drop(columns=['case_id', 'class', is_missing_col], errors='ignore')
    y = label_encoder.transform(df_for_selection['class'])
    del df_for_selection
    
    print(f"  Cases for selection: {len(y):,}")
    
    # --- STAGE 1: Mutual Information Filter (coarse sieve) ---
    print(f"  STAGE 1: Mutual Information filter -> {CONFIG['n_features_stage1']:,} features")
    
    imputer = SimpleImputer(strategy='median')
    X_imputed = imputer.fit_transform(X)
    
    n_stage1 = min(CONFIG['n_features_stage1'], X.shape[1])
    mi_selector = SelectKBest(mutual_info_classif, k=n_stage1)
    mi_selector.fit(X_imputed, y)
    del X_imputed
    
    stage1_features = X.columns[mi_selector.get_support()].tolist()
    print(f"    -> {len(stage1_features):,} features selected")
    del mi_selector
    
    # --- STAGE 2: XGBoost Importance Filter (fine comb) ---
    print(f"  STAGE 2: XGBoost filter -> {CONFIG['n_features_final']} features")
    
    X_stage2 = X[stage1_features]
    del X
    
    xgb_model = XGBClassifier(
        random_state=42,
        n_jobs=-1,
        tree_method='hist',
        subsample=0.8,
        colsample_bytree=0.1
    )
    
    xgb_selector = SelectFromModel(
        xgb_model,
        max_features=CONFIG['n_features_final'],
        prefit=False
    ).fit(X_stage2.values, y)
    del y
    
    final_features = X_stage2.columns[xgb_selector.get_support()].tolist()
    print(f"    -> {len(final_features)} features selected")
    del X_stage2
    
    # --- Reload only selected columns and save ---
    print(f"  Reloading selected columns and saving...")
    columns_to_load = ['case_id', 'class'] + final_features
    df_final = safe_load_parquet(input_file, columns=columns_to_load)
    
    df_final.set_index('case_id', inplace=True)
    df_sorted = df_final.reindex(master_sort_order).reset_index()
    df_sorted.to_parquet(output_file, index=False)
    
    print(f"  -> Saved: {os.path.basename(output_file)}")
    return True

In [ ]:
# --- Load indicators and create master sort order ---
print("Loading indicator file and creating label encoder...")

try:
    indicators_df = pd.read_parquet(CONFIG['indicator_file'])
    master_sort_order = indicators_df['case_id'].sort_values().tolist()
    
    # Create label encoder for class labels
    label_encoder = LabelEncoder()
    label_encoder.fit(indicators_df['class'])
    print(f"  Classes: {list(label_encoder.classes_)}")
    print(f"  Master sort order: {len(master_sort_order)} case_ids")
    
    # Set index for efficient filtering
    indicators_df.set_index('case_id', inplace=True)
    
except FileNotFoundError:
    raise RuntimeError(f"Indicator file not found: {CONFIG['indicator_file']}")

# --- Process all modalities ---
print(f"\n{'='*60}")
print("Starting Two-Stage Funnel Feature Selection")
print(f"{'='*60}")

for modality in CONFIG['modalities']:
    apply_two_stage_selection(modality, indicators_df, label_encoder, master_sort_order)

print(f"\n{'='*60}")
print("Feature selection complete!")
print(f"{'='*60}")

## Stage 3: Output Validation

Scan all output parquet files to verify data quality (NaN presence).

In [ ]:
def scan_directory_for_nans(directory):
    """Scan all parquet files in a directory for NaN values."""
    if not os.path.exists(directory):
        print(f"  Warning: Directory not found: '{directory}'")
        return 0
    
    files = [f for f in os.listdir(directory) if f.endswith('.parquet')]
    if not files:
        print(f"  No Parquet files found in {directory}")
        return 0
    
    total_nans = 0
    for filename in sorted(files):
        file_path = os.path.join(directory, filename)
        try:
            df = safe_load_parquet(file_path)
            nan_count = df.isnull().sum().sum()
            total_nans += nan_count
            status = f"{nan_count:,} NaN values" if nan_count > 0 else "Clean ✓"
            print(f"    {filename}: {status}")
        except Exception as e:
            print(f"    {filename}: Error - {e}")
    
    return total_nans

# --- Scan directories ---
dirs_to_scan = [
    CONFIG['source_dir'],           # Input directory
    CONFIG['output_dir'],           # Output directory
]

print("--- NaN Validation Scan ---\n")

total_nans = 0
for directory in dirs_to_scan:
    print(f"Scanning: {directory}")
    total_nans += scan_directory_for_nans(directory)
    print()

print(f"Total NaN values found: {total_nans:,}")
if total_nans == 0:
    print("✓ All data is clean!")

## Stage 4: Packaging

Copy indicator features and create archive for distribution.

In [ ]:
import shutil

# Copy indicator features to output directory
src_indicator = CONFIG['indicator_file']
dst_indicator = os.path.join(CONFIG['output_dir'], 'indicator_features.parquet')

if os.path.exists(src_indicator):
    shutil.copy(src_indicator, dst_indicator)
    print(f"✓ Copied: indicator_features.parquet")
else:
    print(f"Warning: Indicator file not found at {src_indicator}")

# Create compressed archive
archive_name = 'xgb_reduced.zip'
print(f"Creating archive: {archive_name}")
shutil.make_archive('xgb_reduced', 'zip', CONFIG['output_dir'])
print(f"✓ Archive created: {archive_name}")

# Summary
print(f"\n{'='*60}")
print("Pipeline Complete!")
print(f"{'='*60}")
print(f"Output directory: {CONFIG['output_dir']}/")
print(f"  - 6 modality parquets @ {CONFIG['n_features_final']} features each")
print(f"  - indicator_features.parquet")
print(f"Archive: {archive_name}")